In [ ]:
#pip install django

In [ ]:
#pip install flask

In [ ]:
#pip install streamlit

In [ ]:
import pandas as pd
import numpy as np
#import ML algorithms
from sklearn.ensemble import RandomForestClassifier
#Import model selection and metrics utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

### Import Wisconsin Breast Cancer Dataset {http://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original}

In [ ]:
#pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
breast_cancer_wisconsin_original = fetch_ucirepo(id=15) 
  
# data (as pandas dataframes) 
X = breast_cancer_wisconsin_original.data.features 
y = breast_cancer_wisconsin_original.data.targets 

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
 y.sample(6) #Class Labels --- 2 = benign, 4 = malignant

In [ ]:
X.head(15)

In [ ]:
X.tail(15)

In [ ]:
#Count the number of missing values
X.isnull().sum()

In [ ]:
#Replace missing value with mean

In [ ]:
X.fillna(X['Bare_nuclei'].mean() ,inplace=True)

In [ ]:
#Count the number of missing values
X.isnull().sum()

In [ ]:
#Split dataset on train and test
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3, random_state=42)

In [ ]:
#Instantiate classifier
rf=RandomForestClassifier(n_estimators=150)

In [ ]:
rf.fit(X_train,y_train)

In [ ]:
predictions=rf.predict(X_test)

In [ ]:
accuracy_score(predictions,y_test)

In [ ]:
X.head()

### Store and Retrieve the Model

In [ ]:
import pickle

In [ ]:
pickle.dump(rf, open('saved-models/rf_model.pkl','wb')) #This is the model we will load on the server

In [ ]:
model = pickle.load(open('saved-models/rf_model.pkl','rb'))
print(model.predict([[5,5,9,7,6,1,2,8,3]]))

In [ ]:
X.columns

### Process The Form Data

In [ ]:
from flask import Flask,render_template,request,flash

In [ ]:
app=Flask(__name__) #creates the app to process the user input
#Configure the app
app.logger.setLevel('INFO')
app.secret_key = "amakuru"

#Define the functions that represent the actions that occur as the user interacts with the app.
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['GET','POST'])
def predict():
    if request.method =='POST':
        
        """Clump_thickness', 'Uniformity_of_cell_size',
       'Uniformity_of_cell_shape', 'Marginal_adhesion',
       'Single_epithelial_cell_size', 'Bare_nuclei', 'Bland_chromatin',
       'Normal_nucleoli', 'Mitoses'"""
        try:
            #Retrieve the form inputs
            Clump_thickness=float(request.form['Clump_thickness'])
            Uniformity_of_cell_size=float(request.form['Uniformity_of_cell_size'])
            Uniformity_of_cell_shape=float(request.form['Uniformity_of_cell_shape'])
            Marginal_adhesion=float(request.form['Marginal_adhesion'])
            Single_epithelial_cell_size=float(request.form['Single_epithelial_cell_size']) 
            Bare_nuclei=float(request.form['Bare_nuclei'])
            Bland_chromatin=float(request.form['Bland_chromatin'])
            Normal_nucleoli=float(request.form['Normal_nucleoli'])
            Mitoses=float(request.form['Mitoses'])

            #Consolidate the inputs
            input_args=[Clump_thickness, Uniformity_of_cell_size,\
                       Uniformity_of_cell_shape, Marginal_adhesion,\
                       Single_epithelial_cell_size, Bare_nuclei, \
                       Bland_chromatin, Normal_nucleoli, Mitoses]
            input_arr=np.array(input_args)
            inputs=input_arr.reshape(1,-1)# 1 row, numpy with suggest the number of columns
            #load the saved model
            model = pickle.load(open('saved-models/rf_model.pkl','rb'))
            result=model.predict(inputs)
            #Transform result to human readable
            if int(result)== 2:
                prediction ='Benign'
                color_signal='green'
            else:
                prediction ='Malignant'
                color_signal='red'
                
            #Populate flashed messages
            flash(prediction)
            flash(color_signal)

        except ValueError:
            return "Error: Values not valid."
    return render_template('predict.html',prediction=prediction, color_signal=color_signal)


In [ ]:
if __name__=='__main__':
    app.run(host='localhost', port=1887, debug=False)